In [2]:
!pip install selenium
!apt-get -q update   #Used to handle installation and removal of softwares and libraries
!apt install -yq chromium-chromedriver #ChromeDriver is a separate executable that Selenium WebDriver uses to control Chrome.
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

zsh:1: command not found: apt-get
The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

cp: /usr/lib/chromium-browser/chromedriver: No such file or directory


#### Importing all the necessary Libraries

In [5]:
import sys
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
import os
import copy
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
import sklearn
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time
import re
import seaborn as sns; sns.set_theme()
from sklearn import manifold
%matplotlib inline
import math

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/harshit297/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/harshit297/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/harshit297/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [545]:
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
#WebDriver is a browser automation framework that works with open source APIs. 
#The framework operates by accepting commands, sending those commands to a browser, and interacting with applications.
chrome_options = webdriver.ChromeOptions()
#headless means running chrome with chrome.exe
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

## Getting the links of webpages to scrape

In [548]:
page_url =[ "https://www.beeradvocate.com/beer/top-rated/" ,'https://www.beeradvocate.com/beer/worst/','https://www.beeradvocate.com/beer/trending/']
driver = webdriver.Chrome(ChromeDriverManager().install()) ### Incase of Jupyter Notebook
Links=[]
for url in page_url:
    driver.get(url)
    driver.maximize_window()
    driver.implicitly_wait(10)
    link=driver.find_elements(By.XPATH, value='//*[@id="ba-content"]/table/tbody//td[2]/a')
    for i in link:
        Link=i.get_attribute("href")
        Links.append(Link)

time.sleep(5)
driver.close()

/var/folders/wh/f1gf_vp175112kx_fb3lwg5c0000gn/T/ipykernel_51149/3940004421.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install()) ### Incase of Jupyter Notebook


## Task A and B
### Extract about 5-6k reviews. 

In [553]:
Beer = pd.DataFrame(columns = ['Product','Review','Ratings'])
driver = webdriver.Chrome(ChromeDriverManager().install())
Reviews_final=[]
Ratings_final=[]
for i in Links:

    pageurl=str(i)
    driver.get((pageurl))
    driver.maximize_window()
    driver.implicitly_wait(10)    
    Product =  driver.find_elements(By.CLASS_NAME, value='titleBar')[0]
    Product = Product.text.split('\n')[0]

    user_message = driver.find_elements(By.XPATH, value='//*[@id="rating_fullview_content_2"]/div')
    User_rating = driver.find_elements(By.XPATH, value='//*[@id="rating_fullview_content_2"]/span[5]')

    for msg in user_message:
        Review = msg.text
        Reviews_final.append(Review)
        Beer.loc[len(Beer),'Product'] = Product
    for rat in User_rating:
        Rating = rat.text
        Ratings_final.append(Rating)
Beer['Review']= Reviews_final
Beer['Ratings']= Ratings_final
driver.close()

/var/folders/wh/f1gf_vp175112kx_fb3lwg5c0000gn/T/ipykernel_51149/1801153637.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [555]:
Beer.to_csv('Beer_raw_reviews.csv', index=False)
Beer_read=pd.read_csv('Beer_raw_reviews.csv' )
Beer_df=copy.deepcopy(Beer_read)

In [556]:
Beer_df['Ratings']=Beer_df['Ratings'].apply(lambda x: x.split())

In [557]:
def overall(score):
    x=len(score)
    return score[13]

In [558]:
Beer_df['Ratings']=Beer_df['Ratings'].apply(lambda x: x[len(x)-1])
Beer_df['Ratings'] = pd.to_numeric(Beer_df['Ratings'])
Beer_df.head(5)

In [559]:
Beer_df.to_csv('Beer_reviews.csv', index=False)

# Run from here

## Reading the Reviews file and storing in a dataframe

In [61]:
Beer_df=pd.read_csv('/Users/harshit297/Downloads/Beer_reviews.csv' )

In [62]:
Beer_df.drop_duplicates(inplace=True)
Beer_df.head(5)

,Product,Review,Ratings
0,Kentucky Brunch Brand Stout,I didnt think i was going to give it a perfect...,5.00
1,Kentucky Brunch Brand Stout,So I just read a review that called the legend...,3.75
2,Kentucky Brunch Brand Stout,"2021 vintage, bottle #79\n\nHoly. Fucking. Shi...",4.75
3,Kentucky Brunch Brand Stout,"Celebrating my buddy @Rug with his 1,000th bee...",4.25
4,Kentucky Brunch Brand Stout,"Thick and syrupy pour, mocha head. Aroma is bo...",4.75


In [63]:
Beer_df=Beer_df[Beer_df['Ratings']<=5]

In [64]:
def replacenewline(x):
    return(str(x).replace('\n', ' '))
Beer_df['Review']=Beer_df['Review'].apply(replacenewline)

In [65]:
Beer_df['Review']= Beer_df['Review'].apply(str.lower)
Beer_df.head(3)

,Product,Review,Ratings
0,Kentucky Brunch Brand Stout,i didnt think i was going to give it a perfect...,5.00
1,Kentucky Brunch Brand Stout,so i just read a review that called the legend...,3.75
2,Kentucky Brunch Brand Stout,"2021 vintage, bottle #79 holy. fucking. shit....",4.75


### User inputed attributes

In [66]:
required_attributes_list=['floral','malty','balanced']
required_attributes=' '.join(required_attributes_list)
required_attributes

'floral malty balanced'

## Task C
### Perform a similarity analysis using cosine similarity (without word embeddings) with the 3 attributes specified by the customer and the reviews. 

**Removing the stopwords from the data**

In [67]:
def removing_stopwords(text):
    text_tokens = word_tokenize(text)
    tokens_without_sw =' '.join([word for word in text_tokens if not word in stopwords.words('english')])
    return tokens_without_sw

Beer_df['Review_without_stopwords']=Beer_df['Review'].apply(removing_stopwords)


#### Function to find cosine similarity between two sentences

In [68]:
def cosine(data):
    final_data=[data,required_attributes]
    count_vectorizer = CountVectorizer()
    vector_matrix = count_vectorizer.fit_transform(final_data)
    tokens = count_vectorizer.get_feature_names()
    cosine_similarity_matrix = cosine_similarity(vector_matrix)
    return cosine_similarity_matrix[0][1]

In [69]:
Beer_df['Cosine_Similarity']=Beer_df['Review_without_stopwords'].apply(cosine)

/Users/harshit297/opt/miniconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [70]:
Beer_df.sort_values(by='Cosine_Similarity', ascending=False)

,Product,Review,Ratings,Review_without_stopwords,Cosine_Similarity
4010,Extra Dry,malty little floral yeasty after taste,2.75,malty little floral yeasty taste,0.516398
7173,Centennial IPA,mix between a two hearted ale and an all day i...,4.25,mix two hearted ale day ipa . starts floral ma...,0.436436
7248,Cold Torpedo,pretty decent ipa. balanced with a sneaky 7% a...,3.75,pretty decent ipa . balanced sneaky 7 % abv . ...,0.298142
5264,Red Horse Beer: Extra Strong,"color: light yellow, clear. white foam melts v...",2.25,"color : light yellow , clear . white foam melt...",0.292770
1901,Bad Boy,pours a nice clear golden color with a nice he...,4.75,pours nice clear golden color nice head lacing...,0.288675
...,...,...,...,...,...
2663,Hurricane Malt Liquor,"aroma and flavor is corn/grains, mild fruit an...",2.25,"aroma flavor corn/grains , mild fruit sweet/al...",0.000000
2662,Hurricane Malt Liquor,"(as ""hurricane"" with ""category 5"" on the label...",3.00,( `` hurricane '' `` category 5 '' label abv l...,0.000000
2660,Hurricane Malt Liquor,perfect if you want to get hammered (make sure...,4.75,perfect want get hammered ( make sure get 40oz...,0.000000
2659,Hurricane Malt Liquor,my daily driver! preferred beer in my book. i ...,5.00,daily driver ! preferred beer book . drink 3 2...,0.000000


## Task D
### For every review, perform a sentiment analysis (using VADER)

In [71]:
sid = SentimentIntensityAnalyzer() # Initializing the Analyzer

In [72]:
Beer_df['Scores'] = Beer_df['Review'].apply(lambda review: sid.polarity_scores(review))

In [73]:
Beer_df['Overall_sentiment_score']  = Beer_df['Scores'].apply(lambda score_dict: score_dict['compound'])

In [74]:
Beer_df.head(5)

,Product,Review,Ratings,Review_without_stopwords,Cosine_Similarity,Scores,Overall_sentiment_score
0,Kentucky Brunch Brand Stout,i didnt think i was going to give it a perfect...,5.00,didnt think going give perfect score ..... loo...,0.0,"{'neg': 0.0, 'neu': 0.84, 'pos': 0.16, 'compou...",0.9460
1,Kentucky Brunch Brand Stout,so i just read a review that called the legend...,3.75,"read review called legendary kbbs , “ typical…...",0.0,"{'neg': 0.031, 'neu': 0.775, 'pos': 0.193, 'co...",0.9976
2,Kentucky Brunch Brand Stout,"2021 vintage, bottle #79 holy. fucking. shit....",4.75,"2021 vintage , bottle # 79 holy . fucking . sh...",0.0,"{'neg': 0.085, 'neu': 0.732, 'pos': 0.183, 'co...",0.9927
3,Kentucky Brunch Brand Stout,"celebrating my buddy @rug with his 1,000th bee...",4.25,"celebrating buddy @ rug 1,000th beer review . ...",0.0,"{'neg': 0.061, 'neu': 0.731, 'pos': 0.208, 'co...",0.9476
4,Kentucky Brunch Brand Stout,"thick and syrupy pour, mocha head. aroma is bo...",4.75,"thick syrupy pour , mocha head . aroma boozy m...",0.0,"{'neg': 0.028, 'neu': 0.774, 'pos': 0.198, 'co...",0.8357


## Task E
### Create an evaluation score for each beer that uses both similarity and sentiment scores. E.g., total score  = average of (similarity score + sentiment score) or a multiplicative model.  
### Now recommend 3 products to the customer. 


In [75]:
Beer_scores=Beer_df[['Product','Ratings','Cosine_Similarity','Overall_sentiment_score']]
Beer_scores.head(5)

,Product,Ratings,Cosine_Similarity,Overall_sentiment_score
0,Kentucky Brunch Brand Stout,5.00,0.0,0.9460
1,Kentucky Brunch Brand Stout,3.75,0.0,0.9976
2,Kentucky Brunch Brand Stout,4.75,0.0,0.9927
3,Kentucky Brunch Brand Stout,4.25,0.0,0.9476
4,Kentucky Brunch Brand Stout,4.75,0.0,0.8357


In [76]:
scale = StandardScaler()
scale = sklearn.preprocessing.scale

In [77]:
Beer_scores['Evaluation_score']= (scale(Beer_scores['Cosine_Similarity']) * scale(Beer_scores['Overall_sentiment_score']))
Beer_scores.sort_values(by='Evaluation_score', ascending=False, inplace=True)
Beer_scores.head(5)

/var/folders/_1/bhbzqytn34v356dgx4lslzlw0000gn/T/ipykernel_6181/3418644596.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Beer_scores['Evaluation_score']= (scale(Beer_scores['Cosine_Similarity']) * scale(Beer_scores['Overall_sentiment_score']))
/var/folders/_1/bhbzqytn34v356dgx4lslzlw0000gn/T/ipykernel_6181/3418644596.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Beer_scores.sort_values(by='Evaluation_score', ascending=False, inplace=True)


,Product,Ratings,Cosine_Similarity,Overall_sentiment_score,Evaluation_score
1901,Bad Boy,4.75,0.288675,0.9606,5.936548
5123,Labatt Blue,3.00,0.239474,0.9785,5.084850
7197,Oktoberfest,3.75,0.228665,0.9754,4.806223
7173,Centennial IPA,4.25,0.436436,0.7650,4.797930
1017,Triple Sunshine,4.50,0.233550,0.9541,4.670403


In [78]:
Beer_scores_with_ratings=Beer_scores.groupby(['Product']).agg({'Cosine_Similarity':'mean','Overall_sentiment_score':'mean','Ratings':'mean','Evaluation_score':'mean'})
Beer_scores_with_ratings.reset_index(inplace=True)
Beer_scores_with_ratings[['Product','Evaluation_score']].sort_values(by='Evaluation_score', ascending=False).head(5)

,Product,Evaluation_score
211,Export Lager,1.676044
42,Bad Boy,1.264631
551,Tucher Festbier,0.974265
167,Curiosity One Hundred Twenty Six,0.944315
33,Art,0.908694


#### Insights

The two beers with the closest evaluation scores are Tucher Festbier and Curiosity One Hundred Twenty Six. Two similarities between the two beers is their golden color and hoppiness. Curiosity One Hundred Twenty Six is brewed using entirely concentrated hop products. Similarly, Tucher Festbier is brewed as a lager with a louder presence of hops. The strong hoppy presence in both beers led to a high similarity and sentiment score between the two beers. 

Export Lager and Bad Boy also had similar evaluation scores. Both beers again have a golden color, and they have similar earthy and floral flavors. Along with these flavors, both beers are also noted to have a bitterness to it. Overall, they both have very similar flavors causing high similarity and sentiment scores. 

## Task F
### How would your recommendation change if you use word vectors (e.g., the spaCy package with medium sized pretrained word vectors) instead of the plain vanilla bag-of-words cosine similarity? One way to analyze the difference would be to consider the % of reviews that mention a preferred attribute. E.g., if you recommend a product, what % of its reviews mention an attribute specified by the customer? Do you see any difference across bag-of-words and word vector approaches?

In [79]:
from nltk.tokenize import PunktSentenceTokenizer, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tempfile import NamedTemporaryFile
import shutil
import warnings
import spacy
import csv
import pandas as pd
import nltk
import numpy as np
from nltk.tokenize import PunktSentenceTokenizer, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tempfile import NamedTemporaryFile
import shutil
import warnings
import spacy
import en_core_web_md

In [80]:
nlp = en_core_web_md.load()

In [81]:
required_attributes

'floral malty balanced'

In [82]:
from pandas import DataFrame
from nltk.tokenize import word_tokenize
import string

In [83]:
Beer_df['Review_without_stopwords'].str.cat(sep=' ')


"didnt think going give perfect score ..... looks , smells , tastes , 5 , however , mouth feel means equal look smell taste . pardon sit . mouth feel , well 11. yeh , like twice good anything ive ever . grabs face hands says , 'what went throat ' . tasted familiar -- like real , pure butter real belgium-chocolate , buttermilk pancakes fresh highbush blueberries , real vermont maple syrup madagascar vanilla , served seattle coffee , shot pappy . read review called legendary kbbs , “ typical…white trash. ” kinda made nervous ’ dig white/pale stouts thought , “ could miss white stout ? ! ” well happily guy drinking something else . maybe meant kbbs like mountain dew ; enjoyed european-americans reviewer afraid . bah , cares ? kbbs . looks great poured wine stem . really dark lovely stuff . ’ foam judge beer based foam . yep , ’ present . looks like foam . wine glass supposed enable drinker easily acquire aromas . one heavy adjuncts , result , heavy aromas present overwhelming . lover thin

In [84]:
Beer_df['Review_without_stopwords'] = Beer_df['Review_without_stopwords'].astype(str)


In [85]:
warnings.filterwarnings("ignore", category = DeprecationWarning)

reviews_df = Beer_df[['Review_without_stopwords','Product']].copy()

reviews_df['vector_similarity'] = ''

for index, row in reviews_df.iterrows():
    doc1 = nlp(required_attributes)
    doc2 = nlp(row['Review_without_stopwords'])
    reviews_df.loc[index,'vector_similarity'] = doc1.similarity(doc2)

/var/folders/_1/bhbzqytn34v356dgx4lslzlw0000gn/T/ipykernel_6181/3503983132.py:10: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  reviews_df.loc[index,'vector_similarity'] = doc1.similarity(doc2)


In [90]:
reviews_df.sort_values(by = 'vector_similarity', ascending = False).head(5)

,Review_without_stopwords,Product,vector_similarity
4010,malty little floral yeasty taste,Extra Dry,0.855648
6695,pours clear golden color generous head lacing ...,"Duvel 6,66%",0.834861
599,pours hazy golden color nice head lacing aroma...,Ghost In The Machine - Double Dry-Hopped,0.827467
1901,pours nice clear golden color nice head lacing...,Bad Boy,0.826892
1557,pours nice amber color nice head slight lacing...,Abrasive Ale,0.818845


In [91]:
vector_sim_df = reviews_df.groupby('Product')['vector_similarity'].mean()
vector_sim_df

Product
25th Anniversary Ale                       0.652345
26th Anniversary Imperial IPA              0.624758
333 Premium Export Beer                    0.540940
4th Anniversary                            0.517569
A Deal With The Devil - Double Oak-Aged    0.571942
                                             ...   
Zenne Y Frontera                           0.665927
Zima                                       0.461671
Zombie Dust                                0.547413
Zwanze 2022 - Poivre de Gorilles           0.661961
§ucaba                                     0.538720
Name: vector_similarity, Length: 583, dtype: float64

In [92]:
comparison_df = vector_sim_df.reset_index()
comparison_df = comparison_df.merge(Beer_scores, left_on = 'Product',right_on='Product')

In [93]:
comparison_df=comparison_df.groupby(['Product']).agg({'Cosine_Similarity':'mean','Overall_sentiment_score':'mean','Ratings':'mean','Evaluation_score':'mean','vector_similarity':'mean','vector_similarity':'mean'})


In [94]:
cale = StandardScaler()
scale = sklearn.preprocessing.scale
comparison_df['vector_eval'] = (comparison_df['vector_similarity']*comparison_df['Overall_sentiment_score'])


In [95]:
comparison_df=comparison_df.reset_index()

In [96]:
comparison_df[['Product','vector_similarity','Cosine_Similarity','Overall_sentiment_score','Evaluation_score','vector_eval']].sort_values(by = 'vector_eval', ascending = False).head(5)


,Product,vector_similarity,Cosine_Similarity,Overall_sentiment_score,Evaluation_score,vector_eval
37,Aurelian Lure,0.670478,0.000000,0.958700,-0.300668,0.642788
545,Triple Shot,0.672407,0.000000,0.954275,-0.297426,0.641661
167,Curiosity One Hundred Twenty Six,0.650571,0.055318,0.984400,0.944315,0.640422
219,Flora Plum,0.656627,0.048128,0.937720,0.643348,0.615732
192,Double Dry Hopped Double Mosaic Daydream,0.653880,0.018918,0.938213,0.121001,0.613479


In [97]:
contain_df = Beer_df[['Product','Review_without_stopwords']].copy()


In [98]:
def contains_script(attribute1, attribute2, attribute3,df):
    df['contains ' + attribute1] = 0
    df['contains ' + attribute2] = 0
    df['contains ' + attribute3] = 0
    reviews = list(df['Review_without_stopwords'].values)

    for review in reviews:
        if attribute1 in review:
            df.loc[reviews.index(review), 'contains ' + attribute1] = 1
        else:
            df.loc[reviews.index(review), 'contains ' + attribute1] = 0
        if attribute2 in review:
            df.loc[reviews.index(review), 'contains ' + attribute2] = 1
        else:
            df.loc[reviews.index(review), 'contains ' + attribute2] = 0
        if attribute3 in review:
            df.loc[reviews.index(review), 'contains ' + attribute3] = 1
        else:
            df.loc[reviews.index(review), 'contains ' + attribute3] = 0
    return df

In [99]:
contain_df = contains_script('floral', 'malty', 'balanced',contain_df)


In [100]:
def D(row):
    if row['contains floral'] + row['contains malty'] + row['contains balanced'] == 0:
        val = 0
    else:
        val = 1
    return val

contain_df['contains_attribute'] = contain_df.apply(D, axis = 1)

In [101]:
mean_contain = contain_df.groupby('Product').mean()


In [102]:
comparison_df = comparison_df.merge(mean_contain, left_on = 'Product',right_on='Product')

In [103]:
comparison_df.sort_values(by = 'vector_eval', ascending = False).head(5)

,Product,Cosine_Similarity,Overall_sentiment_score,Ratings,Evaluation_score,vector_similarity,vector_eval,contains floral,contains malty,contains balanced,contains_attribute
37,Aurelian Lure,0.000000,0.958700,4.50000,-0.300668,0.670478,0.642788,0.0,0.0,0.00,0.00
545,Triple Shot,0.000000,0.954275,4.50000,-0.297426,0.672407,0.641661,0.0,0.0,0.00,0.00
167,Curiosity One Hundred Twenty Six,0.055318,0.984400,4.37500,0.944315,0.650571,0.640422,0.0,0.0,0.00,0.00
219,Flora Plum,0.048128,0.937720,4.75000,0.643348,0.656627,0.615732,0.4,0.0,0.40,0.80
192,Double Dry Hopped Double Mosaic Daydream,0.018918,0.938213,4.40625,0.121001,0.653880,0.613479,0.0,0.0,0.25,0.25


We can see the similarity scores for word vectors is greater than the cosine similarity as word vector looks for meaning/essence whereas cosine similarity looks for exact same words.Also a startling observation was 3 out of the 5 beers do not even contain the exact same words we are looking for but have high vector similarity score which is a little bit counterintuitive but can be expected as vector similarity doesnt look for exact same words.When we went inside the reviews and checked the reviews were indeed explaining the attributes well even though they werent mentioning the exact same word we are looking for.Its difficult to say which similarity techniques is recommending better products as there is no sureshot way to verify it.There were no common beers in the top 5 in the cosine and vector analysis.

Top 5 by cosine similarity

In [104]:
comparison_df.sort_values(by = 'Evaluation_score', ascending = False).head(5)

,Product,Cosine_Similarity,Overall_sentiment_score,Ratings,Evaluation_score,vector_similarity,vector_eval,contains floral,contains malty,contains balanced,contains_attribute
211,Export Lager,0.091142,0.919540,4.200000,1.676044,0.649450,0.597196,0.200000,0.200000,0.000000,0.400000
42,Bad Boy,0.057735,0.525280,4.300000,1.264631,0.450054,0.236404,0.200000,0.000000,0.200000,0.200000
551,Tucher Festbier,0.059698,0.777714,3.964286,0.974265,0.625485,0.486449,0.107143,0.178571,0.107143,0.357143
167,Curiosity One Hundred Twenty Six,0.055318,0.984400,4.375000,0.944315,0.650571,0.640422,0.000000,0.000000,0.000000,0.000000
33,Art,0.058926,0.325250,4.812500,0.908694,0.391731,0.127411,0.000000,0.000000,0.250000,0.250000


We can see ratings of the top 5 beers recommended by cosine similarity resulted in low scores, and differed greatly from the vector similarity scores. 

We can see ratings of the top 5 beers recommended by vector similarity are exceptional.We can say this that the vector similarity might have done a better job than cosine similarity but cant be said for certain. 

## Task G
### How would your recommendations differ if you ignored the similarity and feature sentiment scores and simply chose the 3 highest rated products from your entire dataset? Would these products meet the requirements of the user looking for recommendations? Why or why not? Justify your answer with analysis. Use the similarity and sentiment scores as well as overall ratings to answer this question. 

In [106]:
comparison_df.sort_values(by = 'Ratings', ascending = False).head(5)

,Product,Cosine_Similarity,Overall_sentiment_score,Ratings,Evaluation_score,vector_similarity,vector_eval,contains floral,contains malty,contains balanced,contains_attribute
532,The Adjunct Trail - Bourbon Barrel-Aged,0.000000,0.812600,5.000000,-0.193617,0.608499,0.494466,0.0,0.0,0.00,0.00
552,Twice the Daily Serving: Raspberry,0.000000,0.728767,4.833333,-0.132190,0.618009,0.450385,0.0,0.0,0.00,0.00
132,Chemtrailmix - Rye Barrel,0.000000,0.512487,4.812500,0.026283,0.420965,0.215739,0.0,0.0,0.00,0.00
557,Vanilla Bean Assassin,0.010417,0.722575,4.812500,0.113408,0.542212,0.391789,0.0,0.0,0.25,0.25
33,Art,0.058926,0.325250,4.812500,0.908694,0.391731,0.127411,0.0,0.0,0.25,0.25


If we just provide reommendation to users based only on Ratings of beer, The top three beers would be: **The Adjunct Trail - Bourbon Barrel-Aged**, **Twice the Daily Serving: Raspberry**, **Chemtrailmix - Rye Barrel**
But if we look into the Evaluation_score based upon cosine similarity and sentiment scores, They have pretty bad ones with  2 of them having negative values. This means even if the users rated them good, the. beers dont have characteristics required by the users : **'floral malty balanced'**

If we want to recommend beer to users based upon the characteristics required, we should look into the similarity techniques and sentiment scores not just Ratings

The three highest rated products would not justify the requirements the user is looking for. These beers are drastically different, as you can see in their cosine and vector similarity scores are very less,some being less than0. These beers are the oppsite of the golden, smooth beer the user is looking for. They are hoppy, dark, and heavy. They also are aged beers, making them heavy and giving them a different kind of flavor profile, such as cinnamon and raspberry. The user was insterested in dry, light, and crsip beers.

One thing I noticed was that The Adjunct Trail - Bourbon Barrel-Aged beer only had 1 review listed. This is not a strong indication that it was actually one of the highest rated beers. One rating of a 5/5 is not sufficient data for us to jump to this conclusion.

Ignoring similarity and feature sentiment scores results in products without similar features, and not what the user was looking for. There are many different kinds of beers, so one cannot group them based off high ratings and expect the highly rated beers to be similar. They ultimately received these high scores for different reasons, therefore they cannot be compared as similar. 

## Task H
### Using the top four attributes of beer (from word frequency analysis), calculate the lifts between these attributes and any 10 beers in your data. Choose one beer, and find the most similar beer (among the remaining 9) using the lift values. Explain your method. 

In [107]:
Beer_df_tokenized=copy.deepcopy(Beer_df[['Product','Review_without_stopwords']])

In [108]:
def tokenize_word(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text_tokens=tokenizer.tokenize(text)
    return text_tokens

Beer_df_tokenized['Tokenized_Reviews']=Beer_df_tokenized['Review_without_stopwords'].apply(tokenize_word)

In [109]:
def list_words(x,y):
    word_list=[]
    for row in x[y]:
        word_list+=list(set(row))
    return word_list

Word_list=list_words(Beer_df_tokenized,'Tokenized_Reviews')

In [110]:
Word_list_freq=nltk.FreqDist(Word_list)
Word_list_freq

FreqDist({'beer': 3644, 'head': 3574, 'taste': 3218, 'light': 2356, 'like': 2240, 's': 2024, 'good': 1954, 'carbonation': 1866, 'white': 1851, 'one': 1813, ...})

## Chosing the top 4 attributes

In [111]:
top20_attributes=[key for key, item in Word_list_freq.most_common(20)]
top4_attributes=[top20_attributes[x] for x in [2,7,10,12]]
top4_attributes

['taste', 'carbonation', 'sweet', 'aroma']

## Chosing any 10 beers

In [112]:
Beer_Products=Beer_df_tokenized['Product'].unique()
Beer_Brands_10=list(Beer_Products[:10])
Beer_Brands_10

['Kentucky Brunch Brand Stout',
 'Vanilla Bean Assassin',
 'Marshmallow Handjee',
 'Abraxas - Barrel-Aged',
 'Heady Topper',
 'Blessed',
 "Mornin' Delight",
 'King Julius',
 'King JJJuliusss',
 'Zenne Y Frontera']

In [113]:
column=Beer_Brands_10+top4_attributes
beer_feature = pd.DataFrame(columns = column)

def feature_counter(item):
    if features in item:
        return 1
    else:
        return 0
    
def brand_counter(item):
    if beer in item:
        return 1
    else:
        return 0
      
for  beer in Beer_Brands_10:
    beer_feature[beer] = Beer_df_tokenized['Product'].apply(brand_counter)  

for  features in top4_attributes:
    beer_feature[features] = Beer_df_tokenized['Tokenized_Reviews'].apply(feature_counter)

In [114]:
beer_feature_lift = pd.DataFrame(columns = top4_attributes, dtype=float)
length=len(beer_feature)
for i in range(len(Beer_Brands_10)):
    for j in range(len(top4_attributes)):
        
        Confidence=(beer_feature[((beer_feature[Beer_Brands_10[i]]==1) & (beer_feature[top4_attributes[j]]==1 ))][Beer_Brands_10[i]].count())
        Support_i=(beer_feature[((beer_feature[Beer_Brands_10[i]]==1))][Beer_Brands_10[i]].count())
        Support_j=(beer_feature[((beer_feature[top4_attributes[j]]==1))][top4_attributes[j]].count())
        
        if Support_i==0 or Support_j==0:
            Lift=0
        else :
            Lift=(Confidence*length/(Support_i*Support_j))
        beer_feature_lift.loc[Beer_Brands_10[i],top4_attributes[j]]=float(Lift)


beer_feature_lift


,taste,carbonation,sweet,aroma
Kentucky Brunch Brand Stout,0.854646,1.228229,0.258385,0.820970
Vanilla Bean Assassin,0.000000,0.982583,3.100620,0.000000
Marshmallow Handjee,1.012913,0.000000,1.378053,0.973002
Abraxas - Barrel-Aged,0.949606,0.655055,0.689027,0.364876
Heady Topper,0.797669,0.982583,0.620124,1.313552
Blessed,0.651159,0.561476,1.476486,0.625501
Mornin' Delight,1.035934,0.357303,1.879164,0.000000
King Julius,1.424410,0.491292,1.033540,1.368284
King JJJuliusss,0.651159,1.122952,1.181189,0.000000
Zenne Y Frontera,1.139528,2.620222,0.689027,2.189254


In [115]:
beer_feature_lift['Overall_Lift']=(beer_feature_lift['taste']+beer_feature_lift['carbonation']+beer_feature_lift['sweet']+beer_feature_lift['aroma'])/4

In [116]:
beer_feature_lift.sort_values(by='Overall_Lift', ascending=False)

,taste,carbonation,sweet,aroma,Overall_Lift
Zenne Y Frontera,1.139528,2.620222,0.689027,2.189254,1.659507
King Julius,1.424410,0.491292,1.033540,1.368284,1.079381
Vanilla Bean Assassin,0.000000,0.982583,3.100620,0.000000,1.020801
Heady Topper,0.797669,0.982583,0.620124,1.313552,0.928482
Marshmallow Handjee,1.012913,0.000000,1.378053,0.973002,0.840992
Blessed,0.651159,0.561476,1.476486,0.625501,0.828655
Mornin' Delight,1.035934,0.357303,1.879164,0.000000,0.818100
Kentucky Brunch Brand Stout,0.854646,1.228229,0.258385,0.820970,0.790557
King JJJuliusss,0.651159,1.122952,1.181189,0.000000,0.738825
Abraxas - Barrel-Aged,0.949606,0.655055,0.689027,0.364876,0.664641


#### Insights

The beer I chose was the Mornin' Delight. Using the above lift scores for the top four attributes, the most closely related beer is Blessed. The beers are very close with their overall lift scores and have simalr scores on the four attributes. For their carbonation Mornin' Delight has a lift of ~0.36 and Blessed has a score of ~0.56. These are both on the lower side of lift scores compared to the other beers listed above, thus are lightly carbonated beers. As for their sweetness, the two beers are closely related and are on the sweeter side (not as sweet as Vanilla Bean Assassin). I used the lift values to help me determine their relationship and judge what type of beer these two are. From the data and lift scores we can predict the two are lighter sweet beers, very opposite from the heavy Kentucky Brunch Brand Stout. To get a stronger understanding of the beers, it is better to look at the indiviudal lift scores for each attriubte rather than the overall lift score. 